# 02. Data Processing Functions

## 개요
데이터 로딩, 전처리, Dataset/DataLoader 관련 함수들을 정의하는 노트북입니다.

## 주요 기능
- Split 데이터 로딩 및 다운샘플링
- Feature column 추출
- PyTorch Dataset/DataLoader 정의
- Collate 함수들

**주의**: 이 노트북을 실행하기 전에 **01_setup_and_config.ipynb**를 먼저 실행해주세요!


## Data Loading & Preprocessing Functions


In [ ]:
def load_and_downsample_data(file_path, downsample_ratio=2):
    """
    Split 데이터를 로드하고 다운샘플링 수행
    """
    print(f"Loading data from: {file_path}")
    
    # 데이터 로드
    df = pd.read_parquet(file_path, engine="pyarrow")
    print(f"Original shape: {df.shape}")
    
    # clicked == 1 데이터
    clicked_1 = df[df['clicked'] == 1]
    
    # clicked == 0 데이터에서 동일 개수 x downsample_ratio 만큼 무작위 추출
    clicked_0_count = len(clicked_1) * downsample_ratio
    clicked_0 = df[df['clicked'] == 0]
    
    if len(clicked_0) < clicked_0_count:
        # clicked=0 데이터가 부족한 경우 모든 데이터 사용
        print(f"Warning: Not enough clicked=0 data. Using all {len(clicked_0)} samples.")
        sampled_0 = clicked_0
    else:
        sampled_0 = clicked_0.sample(n=clicked_0_count, random_state=42)
    
    # 두 데이터프레임 합치기
    result = pd.concat([clicked_1, sampled_0], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"Downsampled shape: {result.shape}")
    print(f"Clicked=0: {len(result[result['clicked']==0])}, Clicked=1: {len(result[result['clicked']==1])}")
    
    return result

def get_feature_columns(df):
    """피처 컬럼 추출"""
    FEATURE_EXCLUDE = {"clicked", "seq", "ID"}
    return [c for c in df.columns if c not in FEATURE_EXCLUDE]


## PyTorch Dataset Class


In [ ]:
class ClickDataset(Dataset):
    def __init__(self, df, feature_cols, seq_col, target_col=None, has_target=True):
        self.df = df.reset_index(drop=True)
        self.feature_cols = feature_cols
        self.seq_col = seq_col
        self.target_col = target_col
        self.has_target = has_target

        # 비-시퀀스 피처: 전부 연속값으로
        self.X = self.df[self.feature_cols].astype(float).fillna(0).values

        # 시퀀스: 문자열 그대로 보관 (lazy 파싱)
        self.seq_strings = self.df[self.seq_col].astype(str).values

        if self.has_target:
            self.y = self.df[self.target_col].astype(np.float32).values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float)

        # 전체 시퀀스 사용 (빈 시퀀스만 방어)
        s = self.seq_strings[idx]
        if s and s != 'nan':
            try:
                arr = np.fromstring(s, sep=",", dtype=np.float32)
            except:
                arr = np.array([], dtype=np.float32)
        else:
            arr = np.array([], dtype=np.float32)

        if arr.size == 0:
            arr = np.array([0.0], dtype=np.float32)  # 빈 시퀀스 방어

        seq = torch.from_numpy(arr)  # shape (seq_len,)

        if self.has_target:
            y = torch.tensor(self.y[idx], dtype=torch.float)
            return x, seq, y
        else:
            return x, seq


In [ ]:
def collate_fn_train(batch):
    xs, seqs, ys = zip(*batch)
    xs = torch.stack(xs)
    ys = torch.stack(ys)
    seqs_padded = nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=0.0)
    seq_lengths = torch.tensor([len(s) for s in seqs], dtype=torch.long)
    seq_lengths = torch.clamp(seq_lengths, min=1)  # 빈 시퀀스 방지
    return xs, seqs_padded, seq_lengths, ys

def collate_fn_infer(batch):
    xs, seqs = zip(*batch)
    xs = torch.stack(xs)
    seqs_padded = nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=0.0)
    seq_lengths = torch.tensor([len(s) for s in seqs], dtype=torch.long)
    seq_lengths = torch.clamp(seq_lengths, min=1)
    return xs, seqs_padded, seq_lengths

print("Data processing functions loaded successfully!")


## Collate Functions
